# Tarefa 2: Classificação Binária de Empréstimos

Este notebook segue os passos de 'a' até 'i' da avaliação:
1.  Carregar e preparar dados (split, remoção de colunas, encoding, etc.).
2.  Construir um `ColumnTransformer` robusto para pré-processamento.
3.  Usar `imblearn.pipeline.Pipeline` para integrar o pré-processamento, `SMOTE` e o classificador.
4.  Definir e executar `GridSearchCV` para 5 algoritmos diferentes.
5.  Comparar os modelos usando `classification_report` e `roc_auc_score`.
6.  Salvar o pipeline final e completo (com pré-processamento e modelo) usando `joblib`.

### Célula 1: Importação das Bibliotecas

Importamos `pandas` e `numpy` para dados, e diversas ferramentas do `sklearn`: `train_test_split`, pré-processadores (`StandardScaler`, `OneHotEncoder`, `SimpleImputer`), `ColumnTransformer` para organizar o pré-processamento, e as métricas de avaliação. 

Também importamos `SMOTE` (da biblioteca `imbalanced-learn`) para lidar com o desbalanceamento de classe (Instrução 2b) e a versão do `Pipeline` do `imblearn` que é compatível com o SMOTE.

In [ ]:
import pandas as pd
import numpy as np
import warnings
import joblib
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler, OneHotEncoder, FunctionTransformer, LabelEncoder
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.metrics import classification_report, roc_auc_score, confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt

# Pipelines e Modelos
from imblearn.pipeline import Pipeline as ImbPipeline
from imblearn.over_sampling import SMOTE
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import GradientBoostingClassifier

warnings.filterwarnings('ignore')
np.random.seed(42)

### Célula 2: Etapas 2a-f (Carga e Preparação Inicial)

1.  **Carregar Dados:** Baixamos o `loan.csv` do GitHub.
2.  **Remover Colunas (2b):** Removemos as colunas solicitadas.
3.  **Mapear Target (2e):** Convertemos a coluna alvo `Loan_Status` de 'Y'/'N' para 1/0.
4.  **Separar X e y:** Isolamos a variável alvo (`y`) das features (`X`).
5.  **Dividir Treino/Teste (2a):** Dividimos os dados em 80% para treino e 20% para teste (`random_state=42`). Usamos `stratify=y` para garantir que a proporção de classes (Sim/Não) seja a mesma nos dois conjuntos.

In [ ]:
# 1. Carregar Dados
data_url = 'https://raw.githubusercontent.com/josenalde/machinelearning/main/datasets/loan.csv'
df = pd.read_csv(data_url)

# 2. Remover Colunas (Instrução 2b)
cols_to_drop = ['Loan_ID', 'CoapplicantIncome', 'Loan_Amount_Term', 'Credit_History', 'Property_Area']
df = df.drop(columns=cols_to_drop)

# 4. Mapear Target (Instrução 2e)
df['Loan_Status'] = df['Loan_Status'].map({'Y': 1, 'N': 0})
df = df.dropna(subset=['Loan_Status']) # Remover linhas onde o alvo é nulo

# 5. Separar X e y
y = df['Loan_Status']
X = df.drop('Loan_Status', axis=1)

# 6. Separar Treino e Teste (Instrução 2a)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

print("Balanceamento do Target (Treino) antes do SMOTE:")
print(y_train.value_counts(normalize=True))

### Célula 3: Função Auxiliar para 'Dependents'

**Etapa 2f:** A instrução pede para mapear '3+' para '3'. Criamos uma função `map_dependents` que faz exatamente isso. Usaremos `FunctionTransformer` para incluir esta etapa personalizada dentro do pipeline.

In [ ]:
# 3. Mapear 'Dependents' (Instrução 2f)
# Esta transformação será colocada em um FunctionTransformer dentro do pipeline
def map_dependents(X_in):
    # X_in será um DataFrame (ou array) apenas com a coluna 'Dependents'
    X = pd.DataFrame(X_in, columns=['Dependents'])
    X['Dependents'] = X['Dependents'].astype(str).replace('3+', '3')
    return X

### Célula 4: Definição dos Pipelines de Pré-processamento

Esta é a parte central do pré-processamento. Usamos `ColumnTransformer` para aplicar diferentes transformações a diferentes colunas:

1.  **`numeric_transformer` (Etapa 2g):**
    * Para `ApplicantIncome` e `LoanAmount`.
    * Usa `SimpleImputer(strategy='median')` para preencher valores faltantes (NaN) com a mediana.
    * Usa `StandardScaler` para padronizar os dados.

2.  **`categorical_transformer` (Etapas 2c, 2e):**
    * Para `Gender`, `Married`, `Education`, `Self_Employed`.
    * Usa `SimpleImputer(strategy='most_frequent')` para preencher NaNs com a moda (valor mais comum).
    * Usa `OneHotEncoder` para converter as categorias (ex: 'Male'/'Female') em colunas numéricas (0s e 1s).

3.  **`dependents_transformer` (Etapas 2c, 2f):**
    * Apenas para `Dependents`.
    * Imputa NaNs com a moda (2c).
    * Aplica a função `map_dependents` para tratar o '3+' (2f).
    * Usa `OneHotEncoder` para converter '0', '1', '2', '3' em colunas numéricas.

In [ ]:
# 7. Definir Pipelines de Pré-processamento (Instruções 2c, 2e, 2g)

# Features numéricas para escalar (2g)
numeric_features = ['ApplicantIncome', 'LoanAmount']
numeric_transformer = ImbPipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')), # Adicionando imputer para robustez
    ('scaler', StandardScaler())
])

# Features categóricas para imputar com moda (2c) e codificar (2e)
# Usamos OneHotEncoder (em vez de LabelEncoder) pois é o padrão para features nominais em ML
categorical_features = ['Gender', 'Married', 'Education', 'Self_Employed']
categorical_transformer = ImbPipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore', sparse_output=False))
])

# Feature 'Dependents' com tratamento especial (2c, 2f)
dependents_feature = ['Dependents']
dependents_transformer = ImbPipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')), # Imputa com moda (ex: '0')
    ('mapper', FunctionTransformer(map_dependents)), # Mapeia '3+' para '3'
    # Especifica as categorias para garantir a ordem correta no OneHotEncoder
    ('onehot', OneHotEncoder(handle_unknown='ignore', categories=[['0', '1', '2', '3']], sparse_output=False))
])

# Montar o ColumnTransformer
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features),
        ('dep', dependents_transformer, dependents_feature)
    ],
    remainder='passthrough'
)

### Célula 5: Etapa 2h - Definição dos 5 Pipelines e Grids de Busca

Definimos os 5 algoritmos de classificação e os hiperparâmetros que queremos testar para cada um deles. Os nomes no grid (ex: `classifier__C`) referem-se aos passos que definiremos no pipeline da próxima célula (`smote` e `classifier`).

In [ ]:
# Modelos para testar
models_to_search = {
    'LogisticRegression': LogisticRegression(random_state=42, solver='liblinear'),
    'RandomForest': RandomForestClassifier(random_state=42),
    'KNeighbors': KNeighborsClassifier(),
    'SVC': SVC(probability=True, random_state=42),
    'GradientBoosting': GradientBoostingClassifier(random_state=42)
}

# Grids de Hiperparâmetros (reduzidos para velocidade)
param_grids = {
    'LogisticRegression': {
        'classifier__C': [0.1, 1.0, 10],
        'classifier__penalty': ['l1', 'l2']
    },
    'RandomForest': {
        'classifier__n_estimators': [100, 200],
        'classifier__max_depth': [5, 10, None]
    },
    'KNeighbors': {
        'classifier__n_neighbors': [5, 7, 9]
    },
    'SVC': {
        'classifier__C': [0.1, 1.0],
        'classifier__kernel': ['linear', 'rbf']
    },
    'GradientBoosting': {
        'classifier__n_estimators': [100, 200],
        'classifier__learning_rate': [0.05, 0.1]
    }
}

best_estimators = {}

print("Iniciando GridSearch para 5 modelos...")

### Célula 6: Etapa 2h (Execução) - GridSearchCV

Este é o loop principal de treinamento. Para cada um dos 5 modelos:
1.  **Cria um `ImbPipeline`:**
    * `preprocessor`: O `ColumnTransformer` que definimos (trata NaNs, '3+', escala e codifica).
    * `smote`: O `SMOTE` (Instrução 2b) para balancear os dados *apenas no treino* (o pipeline aplica isso internamente em cada fold do CV).
    * `classifier`: O modelo sendo testado (ex: `LogisticRegression`).
2.  **Cria o `GridSearchCV`:** Passamos o pipeline e o grid de parâmetros correspondente. Usamos `cv=5` (validação cruzada de 5 folds) e `scoring='roc_auc'` (como solicitado).
3.  **Executa `grid_search.fit`:** O grid search treina e avalia todas as combinações de parâmetros no conjunto `X_train`, `y_train`.

In [ ]:
for name, model in models_to_search.items():
    print(f"\n--- Buscando para {name} ---")
    
    # Criar o pipeline completo com SMOTE (Instrução 2b)
    pipeline = ImbPipeline(steps=[
        ('preprocessor', preprocessor),
        ('smote', SMOTE(random_state=42)),
        ('classifier', model)
    ])
    
    grid_search = GridSearchCV(
        pipeline, 
        param_grid=param_grids[name], 
        cv=5, 
        scoring='roc_auc', # Comparando por AUC como solicitado
        n_jobs=-1
    )
    
    grid_search.fit(X_train, y_train)
    best_estimators[name] = grid_search.best_estimator_
    
    print(f"Melhor Score (AUC) CV: {grid_search.best_score_:.4f}")
    print(f"Melhores Parâmetros: {grid_search.best_params_}")

### Célula 7: Etapa 2h (Avaliação) - Comparação no Teste

Avaliamos o melhor pipeline (`best_estimator_`) de cada um dos 5 modelos no conjunto de teste (`X_test`, `y_test`), que foi mantido separado durante todo o processo.

Imprimimos o **AUC** e o **Classification Report** (precision, recall, f1-score) para cada um, e plotamos a Matriz de Confusão para análise visual.

In [ ]:
print("\n--- Avaliação dos Melhores Modelos no Conjunto de Teste ---")
best_model_name = None
best_model_auc = 0.0
final_pipeline_to_save = None

for name, model_pipeline in best_estimators.items():
    y_pred = model_pipeline.predict(X_test)
    y_proba = model_pipeline.predict_proba(X_test)[:, 1]
    auc = roc_auc_score(y_test, y_proba)
    
    print(f"\n========== {name} ==========")
    print(f"AUC no Teste: {auc:.4f}")
    print("Classification Report:")
    print(classification_report(y_test, y_pred))
    
    cm = confusion_matrix(y_test, y_pred)
    disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=['Negado', 'Aprovado'])
    disp.plot(cmap=plt.cm.Blues)
    plt.title(f"Matriz de Confusão - {name}")
    plt.show()
    
    if auc > best_model_auc:
        best_model_auc = auc
        best_model_name = name
        final_pipeline_to_save = model_pipeline

### Célula 8: Etapa 2i - Salvando o Modelo Final

Selecionamos o pipeline que teve o melhor AUC no conjunto de teste e o salvamos em `loan_pipeline.joblib`. Este arquivo único contém todas as etapas: pré-processamento (imputação, scaling, one-hot-encoding) e o modelo classificador já treinado.

In [ ]:
model_filename = 'loan_pipeline.joblib'
joblib.dump(final_pipeline_to_save, model_filename)

print(f"\n--- Etapa 2i Concluída ---")
print(f"Melhor modelo: {best_model_name} (AUC: {best_model_auc:.4f})")
print(f"Pipeline final salvo em: {model_filename}")